In [1]:
import pandas as pd
from functools import partial
from pathlib import Path

import sys
sys.path.insert(0, '../..')
from src.utils.latex import show_latex, TABLES

# Load data

In [2]:
path = Path('../../data')

cov_2020 = pd.read_pickle(path / '2020_raw/ICD_U07.1/notes_[U07.1]_2020_q1_q2_q3.pkl')
kwd_cov_2020 = pd.read_pickle(path / 'keyword_results/cov_2020_kwd_v1.pkl')
annotated = pd.read_csv(path / 'annotated_notes_ids.csv', dtype={'MDN': str, 'NotitieID': str})

In [3]:
df = cov_2020.drop_duplicates(subset=['institution', 'MDN', 'NotitieID', 'all_text'], keep='first'
)[['institution', 'MDN', 'NotitieID']
].merge(kwd_cov_2020, on=['institution', 'MDN', 'NotitieID'], how='left'
).merge(annotated.iloc[:,:-1], on=['institution', 'MDN', 'NotitieID'], how='left')

# Stats

In [4]:
domains = ['ENR', 'ATT', 'STM', 'ADM', 'INS', 'MBW', 'FAC', 'BER']
matched_domains = [f"matched_{domain}" for domain in domains]
count_domains = [f"n_{domain}" for domain in domains]

def op_count(df, domain):
    "Number of matches for `domain` keywords."
    return df[domain].apply(lambda x: len(x) if x==x else 0)

def op_bool(df, domain):
    "Are there any matches for `domain` keywords (boolean)."
    return df[domain].fillna(0).apply(bool)

ops_count = {f"n_{domain}":partial(op_count, domain=domain) for domain in domains}
ops_bool = {f"matched_{domain}":partial(op_bool, domain=domain) for domain in domains}

In [5]:
results = df.assign(**ops_count, **ops_bool
).assign(
    kwd_match=lambda df: df[domains].any(axis=1),
    batch=lambda df: df.batch.fillna('not annotated'),
    n_domains=lambda df: df[matched_domains].sum(axis=1)
)

### Overview

In [6]:
caption = "Num. notes with/without keyword matches in the COVID data" 
label = "kwd_covid_overview"

results.pivot_table(
    index=['batch','institution',],
    columns=['kwd_match'],
    values='NotitieID',
    aggfunc='count',
    margins=True,
    margins_name='Totals',
).pipe(show_latex, caption, label)

kwd_match                  False    True  Totals
batch         institution                       
not annotated amc          4,765  17,014  21,779
              vumc         4,837  17,101  21,938
pilot         amc             34     919     953
              vumc            34     727     761
Totals                     9,670  35,761  45,431

### Stats by domain (non-annotated notes only)

In [8]:
q_no_ann = "batch == 'not annotated'"
q_match = "n_domains > 0"

In [9]:
caption = "Domains overview: num. notes with at least one keyword match" 
label = "kwd_covid_matched_domains"

results.query(q_no_ann).pivot_table(
    index=['institution',],
    values=matched_domains,
    aggfunc='sum',
    margins=True,
    margins_name='Totals',
).T.sort_values('Totals', ascending=False).pipe(show_latex, caption, label)

institution,amc,vumc,Totals
matched_ADM,"12,172","11,558","23,730"
matched_MBW,"9,371","9,389","18,760"
matched_BER,"5,717","5,032","10,749"
matched_FAC,"4,526","4,567","9,093"
matched_ENR,"3,990","4,019","8,009"
matched_INS,"2,904","3,630","6,534"
matched_STM,"2,813","3,067","5,880"
matched_ATT,"2,559","2,498","5,057"


In [10]:
caption = " Mean/median/max num. keyword matches per domain" 
label = "kwd_covid_kwd_per_dom"

dfs = [results.query(q_no_ann).query(f"n_{dom} > 0")[f"n_{dom}"].agg(['mean', 'median', 'max']) for dom in domains]
pd.concat(dfs, keys=domains).unstack(1).sort_values('mean', ascending=False).round(2
).astype({
    'median': int,
    'max': int,
}).pipe(show_latex, caption, label)

,mean,median,max
ADM,3.32,2,39
MBW,3.1,2,134
BER,2.06,1,22
INS,1.98,1,23
FAC,1.87,1,27
ENR,1.84,1,13
STM,1.56,1,31
ATT,1.39,1,13


In [11]:
caption = "Number of matched domains per note" 
label = "kwd_covid_n_matched domains"

results.query(q_no_ann).query(q_match).pivot_table(
    columns=['institution',],
    index=['n_domains'],
    aggfunc='count',
    values='NotitieID',
    margins=True,
    margins_name='Totals',
).pipe(show_latex, caption, label)

institution,amc,vumc,Totals
n_domains,,,
1,"5,107","5,133","10,240"
2,"4,418","4,528","8,946"
3,"3,323","3,461","6,784"
4,"2,127","2,047","4,174"
5,"1,148","1,084","2,232"
6,501,484,985
7,234,236,470
8,156,128,284
Totals,"17,014","17,101","34,115"


In [12]:
prefix = 'kwd_covid'
for idx, table in enumerate(TABLES):
    with open(f'./tables/{prefix}_{idx}.tex', 'w', encoding='utf8') as f:
        f.write(table)